## Data Wrangling

The Dataset we have is in unstructured format (.txt file). Therefore, we first need to structure it and convert it into a format which is easier to work on.

In [11]:
with open("Datasets/Digital Nomad Travel Logs.txt") as file:
    data = file.readlines()

data

['{"README":["Please mention and link back to Nomad List when you use this data","https:\\/\\/nomadlist.com\\/graph","Last updated 6 hours ago"],"lisbon-portugal":{"porto-portugal":123,"madrid-spain":65,"london-united-kingdom":56,"barcelona-spain":50,"paris-france":40,"amsterdam-netherlands":40,"berlin-germany":36,"madeira-portugal":22,"ericeira-portugal":21,"new-york-city-ny-united-states":21},"prague-czechia":{"berlin-germany":91,"budapest-hungary":70,"london-united-kingdom":32,"amsterdam-netherlands":30,"paris-france":25,"barcelona-spain":23,"krakow-poland":21,"munich-germany":19,"warsaw-poland":17,"new-york-city-ny-united-states":16},"las-palmas-canary-islands-spain":{"london-united-kingdom":16,"madrid-spain":15,"barcelona-spain":13,"tenerife-canary-islands-spain":13,"lisbon-portugal":8,"amsterdam-netherlands":5,"valencia-spain":5,"paris-france":4,"casablanca-morocco":3,"porto-portugal":3},"berlin-germany":{"prague-czechia":111,"amsterdam-netherlands":91,"london-united-kingdom":86,

## String Dictionary to Dictionary

As we can see the data currently is a list with only 1 element which is a string dictionary. Our task for now is to convert that string dictionary into a dictionary. For this we will use the json package of Python

In [16]:
import json

data_dict = json.loads(data[0])
data_dict

{'README': ['Please mention and link back to Nomad List when you use this data',
  'https://nomadlist.com/graph',
  'Last updated 6 hours ago'],
 'lisbon-portugal': {'porto-portugal': 123,
  'madrid-spain': 65,
  'london-united-kingdom': 56,
  'barcelona-spain': 50,
  'paris-france': 40,
  'amsterdam-netherlands': 40,
  'berlin-germany': 36,
  'madeira-portugal': 22,
  'ericeira-portugal': 21,
  'new-york-city-ny-united-states': 21},
 'prague-czechia': {'berlin-germany': 91,
  'budapest-hungary': 70,
  'london-united-kingdom': 32,
  'amsterdam-netherlands': 30,
  'paris-france': 25,
  'barcelona-spain': 23,
  'krakow-poland': 21,
  'munich-germany': 19,
  'warsaw-poland': 17,
  'new-york-city-ny-united-states': 16},
 'las-palmas-canary-islands-spain': {'london-united-kingdom': 16,
  'madrid-spain': 15,
  'barcelona-spain': 13,
  'tenerife-canary-islands-spain': 13,
  'lisbon-portugal': 8,
  'amsterdam-netherlands': 5,
  'valencia-spain': 5,
  'paris-france': 4,
  'casablanca-morocco': 

In [17]:
# Deleting 'README' key from the dictionary

del data_dict['README']
data_dict

{'lisbon-portugal': {'porto-portugal': 123,
  'madrid-spain': 65,
  'london-united-kingdom': 56,
  'barcelona-spain': 50,
  'paris-france': 40,
  'amsterdam-netherlands': 40,
  'berlin-germany': 36,
  'madeira-portugal': 22,
  'ericeira-portugal': 21,
  'new-york-city-ny-united-states': 21},
 'prague-czechia': {'berlin-germany': 91,
  'budapest-hungary': 70,
  'london-united-kingdom': 32,
  'amsterdam-netherlands': 30,
  'paris-france': 25,
  'barcelona-spain': 23,
  'krakow-poland': 21,
  'munich-germany': 19,
  'warsaw-poland': 17,
  'new-york-city-ny-united-states': 16},
 'las-palmas-canary-islands-spain': {'london-united-kingdom': 16,
  'madrid-spain': 15,
  'barcelona-spain': 13,
  'tenerife-canary-islands-spain': 13,
  'lisbon-portugal': 8,
  'amsterdam-netherlands': 5,
  'valencia-spain': 5,
  'paris-france': 4,
  'casablanca-morocco': 3,
  'porto-portugal': 3},
 'berlin-germany': {'prague-czechia': 111,
  'amsterdam-netherlands': 91,
  'london-united-kingdom': 86,
  'budapest-h

## Structuring the data

Now, we have a dictionary in which key of every element is the place from where the digital nomads travelled and the value contains a dictionary which shows the cities they travelled to and the number of times they travelled.
Now we have to convert this data into 3 python lists with each having values, the city from where the digital nomads travelled, the city where they travelled to, and the number of times they travelled respectively

In [20]:
from_cities = []
to_cities = []
number_of_trips = []

{'porto-portugal': 123,
 'madrid-spain': 65,
 'london-united-kingdom': 56,
 'barcelona-spain': 50,
 'paris-france': 40,
 'amsterdam-netherlands': 40,
 'berlin-germany': 36,
 'madeira-portugal': 22,
 'ericeira-portugal': 21,
 'new-york-city-ny-united-states': 21}